In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import DataLoader, TensorDataset

# IMDB dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [2]:
#using my own implementation similar to the textbook:
#this was done as I could not find functions like load_sequential, etc.

# pad sequences are of max length of 500 (this is what load_sequential() does from book
maxlen = 500
train_data = pad_sequences(train_data, maxlen=maxlen)
test_data = pad_sequences(test_data, maxlen=maxlen)

# Convert to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.float32)

# this acts similar to load_tensor() function from the book
#i changed the batch size and made it smaller than 512 since it yielded better performance
batch_size = 32
#nowe we can make the actual datasets
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)
#atual data to be used by loading
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class SentimentClassifier(nn.Module):
    #given by the book , we can use this 
    def __init__(self, vocab_size, embedding_dim=32, hidden_units=16, dropout_rate=0.0):
        super(SentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * maxlen, hidden_units)
        self.fc2 = nn.Linear(hidden_units, hidden_units)
        self.output = nn.Linear(hidden_units, 1)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), -1)  # the book uses torch.flatten(val) - but val = x so its the same
        #1) hidden layer 1 with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        #2) hidden layer 2 with dropout
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        #above we implemented the two hidden layers with the dropout afterwards
        x = torch.sigmoid(self.output(x))
        return x

In [4]:
from torch.optim import Adam

def train_model(model, train_loader, test_loader, epochs=5, learning_rate=0.001):
    #this is functioning similar to the SimpleModule library which was given
    # adam optimizer
    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
    model.train()
    
    #iterating for each epoch 
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

    evaluate_model(model, test_loader)

In [5]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    #basic evaluation as it examines whether or not predictions match labels
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions = (outputs.squeeze() > 0.5).float()
            correct += (predictions == labels).sum().item() 
            total += labels.size(0)
    #measuring performance
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [6]:
model = SentimentClassifier(vocab_size=10000, hidden_units=16, dropout_rate=0.0)
train_model(model, train_loader, test_loader)

Epoch 1/5, Loss: 0.6904
Epoch 2/5, Loss: 0.4912
Epoch 3/5, Loss: 0.2572
Epoch 4/5, Loss: 0.1629
Epoch 5/5, Loss: 0.1113
Test Accuracy: 81.62%


In [7]:
model = SentimentClassifier(vocab_size=10000, hidden_units=32, dropout_rate=0.0)
train_model(model, train_loader, test_loader)

Epoch 1/5, Loss: 0.6758
Epoch 2/5, Loss: 0.4264
Epoch 3/5, Loss: 0.2476
Epoch 4/5, Loss: 0.1657
Epoch 5/5, Loss: 0.1118
Test Accuracy: 79.14%


In [8]:
model = SentimentClassifier(vocab_size=10000, hidden_units=32, dropout_rate=0.3)
train_model(model, train_loader, test_loader)

Epoch 1/5, Loss: 0.6943
Epoch 2/5, Loss: 0.6719
Epoch 3/5, Loss: 0.6386
Epoch 4/5, Loss: 0.6178
Epoch 5/5, Loss: 0.6024
Test Accuracy: 53.00%


In [10]:
model = SentimentClassifier(vocab_size=10000, hidden_units=64, dropout_rate=0.0)
train_model(model, train_loader, test_loader)

Epoch 1/5, Loss: 0.6968
Epoch 2/5, Loss: 0.4988
Epoch 3/5, Loss: 0.2686
Epoch 4/5, Loss: 0.1778
Epoch 5/5, Loss: 0.1249
Test Accuracy: 80.72%


In [11]:
model = SentimentClassifier(vocab_size=10000, hidden_units=64, dropout_rate=0.3)
train_model(model, train_loader, test_loader)

Epoch 1/5, Loss: 0.6998
Epoch 2/5, Loss: 0.6746
Epoch 3/5, Loss: 0.6433
Epoch 4/5, Loss: 0.6285
Epoch 5/5, Loss: 0.6194
Test Accuracy: 51.20%


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from ISLP.torch.imdb import load_sparse
from ISLP.torch import summary_plot

# Step 1: Load the Sparse Data
((X_train, Y_train), (X_valid, Y_valid), (X_test, Y_test)) = load_sparse(
    validation=2000,
    random_state=0,
    root="data/IMDB"
)

# Step 2: Compute Regularization Path
lam_max = np.abs(X_train.T @ (Y_train - Y_train.mean())).max()
lam_val = lam_max * np.exp(np.linspace(np.log(1), np.log(1e-4), 50))

# Step 3: Fit Lasso Logistic Regression
logit = LogisticRegression(
    penalty="l1",
    C=1 / lam_max,
    solver="liblinear",
    warm_start=True,
    fit_intercept=True
)

coefs, intercepts = [], []
for l in lam_val:
    logit.C = 1 / l
    logit.fit(X_train, Y_train)
    coefs.append(logit.coef_.copy())
    intercepts.append(logit.intercept_)

# Squeeze extraneous dimensions
coefs = np.squeeze(coefs)
intercepts = np.squeeze(intercepts)

# Step 4: Plot Lasso Accuracy
fig, axes = plt.subplots(1, 2, figsize=(16, 8), sharey=True)

for (X_, Y_, data_, color) in zip(
    [X_train, X_valid, X_test],
    [Y_train, Y_valid, Y_test],
    ["Training", "Validation", "Test"],
    ["black", "red", "blue"]
):
    linpred_ = X_ @ coefs.T + intercepts[None, :]
    label_ = np.array(linpred_ > 0)
    accuracy_ = np.array([accuracy_score(Y_, l) for l in label_.T])
    axes[0].plot(
        -np.log(lam_val / X_train.shape[0]),
        accuracy_,
        ".--",
        color=color,
        markersize=13,
        linewidth=2,
        label=data_
    )
axes[0].legend()
axes[0].set_xlabel(r"$-\log(\lambda)$", fontsize=20)
axes[0].set_ylabel("Accuracy", fontsize=20)

# Step 5: Neural Network Accuracy
imdb_results = pd.read_csv(imdb_logger.experiment.metrics_file_path)
summary_plot(
    imdb_results,
    axes[1],
    col="accuracy",
    ylabel="Accuracy"
)
axes[1].set_xticks(np.linspace(0, 30, 7).astype(int))
axes[1].set_ylabel("Accuracy", fontsize=20)
axes[1].set_xlabel("Epoch", fontsize=20)
axes[1].set_ylim([0.5, 1])
axes[1].axhline(test_results[0]["test_accuracy"], color="blue", linestyle="--", linewidth=3)

plt.tight_layout()
plt.show()

# Step 6: Cleanup
del (
    imdb_model,
    imdb_trainer,
    imdb_logger,
    imdb_dm,
    imdb_train,
    imdb_test
)

RuntimeError: operator torchvision::nms does not exist